In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, Comment
import time
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO
import io
import copy

## Data Scrape Index: (1950-Current Season)

### All data was scraped from basketballreference.com
https://www.basketball-reference.com/ 

##### Section 1 | Player Data: (6 csv's)
- RS | Per Game | Player
- RS | Advanced | Player
- RS | Totals | Player
- PO | Per Game | Player
- PO | Advanced | Player
- PO | Totals | Player

- Time: 38 minutes

##### Section 2 | All Awards Voting/Teams (Player/Coaches): (2 csv's)
- MVP voting | Player
- ROY voting | Player
- DPOY voting | Player
- SMOY voting | Player
- MIP voting | Player
- CPOY voting | Player
- All-NBA | Player
- All-Defense | Player
- All-Rookie | Player
- COY voting | Coach

- Time: 366 minutes

##### Section 3 | Team Data: (6 csv's)
- RS | Per Game | Team
- RS | Opp Per Game | Team
- RS | Advanced | Team
- PO | Advaned | Team
- RS | Schedule/Results | Team (play-in games NOT counted)
- PO | Schedule/Results | Team (play-in games NOT counted)

- Time: 38 minutes

##### Section 4  (Team and Coaches): (3 csv's)
- RS | Pre-Season Odds | Team
- RS | Expanded Standings | Team
- Season | Coaches

- Time: 60 minutes

##### Team/Player Indexs | NOT SCRAPED | Needed to be updated Yearly.
- custom_team_season_index | contains data relating to each seasons team, conference, playoff_seed, playoff matchups, team_id, league, and team abbreviations.
- custom_team_franchise_index | contains data unique to every NBA Franchise ever (has id for each team, think about it if a team change names its still the franchise, example Seattle SuperSonics and Oklahoma City Thunder have the same team id.

##### Total Files and Scrape:
- 17 scrape files
- 2 index manually updation files
- total files 19
- total scrape time = 8.37 hours or 502 minutes

In [3]:
#set year range

start = 1950 #
end = 2025 #minus 1, so if you type 2025, it actually means = 2024

seasons_list = [str(year) for year in range(start, end)]

### RS | Per Game | Player

In [41]:

def scrape_season(season):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_per_game.html'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'per_game_stats'})

        if table:
            html_str = str(table)
            df = pd.read_html(StringIO(html_str))[0]

            df['Season'] = season

            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"No table found for {season}")
            return None
    else:
        print(f"Failed to retrieve data for {season}")
        return None

    time.sleep(4)

    return all_data

def scrape_all_seasons(seasons):
    all_data = pd.DataFrame()

    total_seasons = len(seasons)

    for i, season in enumerate(reversed(seasons), 1):
        data = scrape_season(season)

        if data is not None:
            all_data = pd.concat([all_data, data], ignore_index=True)

        completion_percentage = (i / total_seasons) * 100
        sys.stdout.write(f"\rScraping: [{'#' * int(completion_percentage // 2)}{' ' * (50 - int(completion_percentage // 2))}] {completion_percentage:.2f}%")
        sys.stdout.flush()

    sys.stdout.flush()

    all_data = all_data[['Season'] + [col for col in all_data.columns if col != 'Season']]
    all_data = all_data[~all_data['Player'].isin(all_data.columns)]

    return all_data

seasons_list = [str(year) for year in range(start, end)]

result = scrape_all_seasons(seasons_list)

result['Player'] = result['Player'].str.replace('*', '')

RS_Per_Game_Player = result

#save
RS_Per_Game_Player.to_csv('RS_Per_Game_Player.csv', index=False, encoding="utf-8-sig")

Scraping: [##################################################] 100.00%

### RS | Advanced | Player

In [44]:

def scrape_season(season):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_advanced.html'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'advanced_stats'})

        if table:
            html_str = str(table)
            df = pd.read_html(StringIO(html_str))[0]

            df['Season'] = season

            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"No table found for {season}")
            return None
    else:
        print(f"Failed to retrieve data for {season}")
        return None

    time.sleep(4)

    return all_data

def scrape_all_seasons(seasons):
    all_data = pd.DataFrame()

    total_seasons = len(seasons)

    for i, season in enumerate(reversed(seasons), 1):
        data = scrape_season(season)

        if data is not None:
            all_data = pd.concat([all_data, data], ignore_index=True)

        completion_percentage = (i / total_seasons) * 100
        sys.stdout.write(f"\rScraping: [{'#' * int(completion_percentage // 2)}{' ' * (50 - int(completion_percentage // 2))}] {completion_percentage:.2f}%")
        sys.stdout.flush()

    sys.stdout.flush()

    all_data = all_data[['Season'] + [col for col in all_data.columns if col != 'Season']]
    all_data = all_data[['Season'] + [col for col in all_data.columns if col not in ['Season']]]
    all_data = all_data[~all_data['Player'].isin(all_data.columns)]

    return all_data

seasons_list = [str(year) for year in range(start, end)]

result = scrape_all_seasons(seasons_list)

result = result.loc[:, ~result.columns.str.contains('^Unnamed')]

result['Player'] = result['Player'].str.replace('*', '')

RS_Advanced_Player = result

#save
RS_Advanced_Player.to_csv('RS_Advanced_Player.csv', index=False, encoding="utf-8-sig")

Scraping: [##################################################] 100.00%

### RS | Totals | Player

In [45]:


def scrape_season(season):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_totals.html'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'totals_stats'})

        if table:
            html_str = str(table)
            df = pd.read_html(StringIO(html_str))[0]

            df['Season'] = season

            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"No table found for {season}")
            return None
    else:
        print(f"Failed to retrieve data for {season}")
        return None

    time.sleep(4)

    return all_data

def scrape_all_seasons(seasons):
    all_data = pd.DataFrame()

    total_seasons = len(seasons)

    for i, season in enumerate(reversed(seasons), 1):
        data = scrape_season(season)

        if data is not None:
            all_data = pd.concat([all_data, data], ignore_index=True)

        completion_percentage = (i / total_seasons) * 100
        sys.stdout.write(f"\rScraping: [{'#' * int(completion_percentage // 2)}{' ' * (50 - int(completion_percentage // 2))}] {completion_percentage:.2f}%")
        sys.stdout.flush()

    sys.stdout.flush()

    all_data = all_data[['Season'] + [col for col in all_data.columns if col != 'Season']]
    all_data = all_data[['Season'] + [col for col in all_data.columns if col not in ['Season']]]
    all_data = all_data[~all_data['Player'].isin(all_data.columns)]

    return all_data

seasons_list = [str(year) for year in range(start, end)]

result = scrape_all_seasons(seasons_list)

result = result.loc[:, ~result.columns.str.contains('^Unnamed')]

result['Player'] = result['Player'].str.replace('*', '')

RS_Totals_Player = result

#save
RS_Totals_Player.to_csv('RS_Totals_Player.csv', index=False, encoding="utf-8-sig")

Scraping: [##################################################] 100.00%

### PO | Per Game | Player

In [46]:
def scrape_season(season):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/playoffs/NBA_{season}_per_game.html'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'per_game_stats'})

        if table:
            html_str = str(table)
            df = pd.read_html(StringIO(html_str))[0]

            df['Season'] = season

            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"No table found for {season}")
            return None
    else:
        print(f"Failed to retrieve data for {season}")
        return None

    time.sleep(4)

    return all_data

def scrape_all_seasons(seasons):
    all_data = pd.DataFrame()

    total_seasons = len(seasons)

    for i, season in enumerate(reversed(seasons), 1):
        data = scrape_season(season)

        if data is not None:
            all_data = pd.concat([all_data, data], ignore_index=True)

        completion_percentage = (i / total_seasons) * 100
        sys.stdout.write(f"\rScraping: [{'#' * int(completion_percentage // 2)}{' ' * (50 - int(completion_percentage // 2))}] {completion_percentage:.2f}%")
        sys.stdout.flush()

    sys.stdout.flush()

    all_data = all_data[['Season'] + [col for col in all_data.columns if col != 'Season']]
    all_data = all_data[~all_data['Player'].isin(all_data.columns)]

    return all_data

seasons_list = [str(year) for year in range(start, end)]

result = scrape_all_seasons(seasons_list)

result['Player'] = result['Player'].str.replace('*', '')

PO_Per_Game_Player = result

#save
PO_Per_Game_Player.to_csv('PO_Per_Game_Player.csv', index=False, encoding="utf-8-sig")

Scraping: [##################################################] 100.00%

### PO | Advanced | Player

In [47]:
def scrape_season(season):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/playoffs/NBA_{season}_advanced.html'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'advanced_stats'})

        if table:
            html_str = str(table)
            df = pd.read_html(StringIO(html_str))[0]

            df['Season'] = season

            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"No table found for {season}")
            return None
    else:
        print(f"Failed to retrieve data for {season}")
        return None

    time.sleep(4)

    return all_data

def scrape_all_seasons(seasons):
    all_data = pd.DataFrame()

    total_seasons = len(seasons)

    for i, season in enumerate(reversed(seasons), 1):
        data = scrape_season(season)

        if data is not None:
            all_data = pd.concat([all_data, data], ignore_index=True)

        completion_percentage = (i / total_seasons) * 100
        sys.stdout.write(f"\rScraping: [{'#' * int(completion_percentage // 2)}{' ' * (50 - int(completion_percentage // 2))}] {completion_percentage:.2f}%")
        sys.stdout.flush()

    sys.stdout.flush()

    all_data = all_data[['Season'] + [col for col in all_data.columns if col != 'Season']]
    all_data = all_data[~all_data['Player'].isin(all_data.columns)]

    return all_data

seasons_list = [str(year) for year in range(start, end)]

result = scrape_all_seasons(seasons_list)

result = result.loc[:, ~result.columns.str.contains('^Unnamed')]

result['Player'] = result['Player'].str.replace('*', '')

PO_Advanced_Player = result

#save
PO_Advanced_Player.to_csv('PO_Advanced_Player.csv', index=False, encoding="utf-8-sig")

Scraping: [##################################################] 100.00%

### PO | Totals | Player

In [48]:

def scrape_season(season):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/playoffs/NBA_{season}_totals.html'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'totals_stats'})

        if table:
            html_str = str(table)
            df = pd.read_html(StringIO(html_str))[0]

            df['Season'] = season

            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"No table found for {season}")
            return None
    else:
        print(f"Failed to retrieve data for {season}")
        return None

    time.sleep(4)

    return all_data

def scrape_all_seasons(seasons):
    all_data = pd.DataFrame()

    total_seasons = len(seasons)

    for i, season in enumerate(reversed(seasons), 1):
        data = scrape_season(season)

        if data is not None:
            all_data = pd.concat([all_data, data], ignore_index=True)

        completion_percentage = (i / total_seasons) * 100
        sys.stdout.write(f"\rScraping: [{'#' * int(completion_percentage // 2)}{' ' * (50 - int(completion_percentage // 2))}] {completion_percentage:.2f}%")
        sys.stdout.flush()

    sys.stdout.flush()

    all_data = all_data[['Season'] + [col for col in all_data.columns if col != 'Season']]
    all_data = all_data[['Season'] + [col for col in all_data.columns if col not in ['Season']]]
    all_data = all_data[~all_data['Player'].isin(all_data.columns)]

    return all_data

seasons_list = [str(year) for year in range(start, end)]

result = scrape_all_seasons(seasons_list)

result = result.loc[:, ~result.columns.str.contains('^Unnamed')]

result['Player'] = result['Player'].str.replace('*', '')

PO_Totals_Player = result

#save
PO_Totals_Player.to_csv('PO_Totals_Player.csv', index=False, encoding="utf-8-sig")

Scraping: [##################################################] 100.00%

### All Award Voting | Player | Coaches

In [49]:
def scrape_season_table(season, table_id):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/awards/awards_{season}.html'

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)

    try:
        table = driver.find_element(By.ID, table_id)
        table_html = table.get_attribute('outerHTML')
        driver.quit()

        df = pd.read_html(StringIO(table_html), header=[1])[0]

        df['Season'] = season
        df['award_type'] = table_id

        all_data = pd.concat([all_data, df], ignore_index=True)
    except Exception as e:
        print(f"No table found for {table_id} in {season}")
        print(e)
        driver.quit()
        return None

    time.sleep(5)

    return all_data

def scrape_all_seasons_tables(seasons, table_ids):
    all_data = pd.DataFrame()

    total_tasks = len(seasons) * len(table_ids)
    task_count = 0

    for season in reversed(seasons):
        for table_id in table_ids:
            data = scrape_season_table(season, table_id)

            if data is not None:
                all_data = pd.concat([all_data, data], ignore_index=True)

            task_count += 1
            completion_percentage = (task_count / total_tasks) * 100
            sys.stdout.write(f"\rScraping: [{'#' * int(completion_percentage // 2)}{' ' * (50 - int(completion_percentage // 2))}] {completion_percentage:.2f}%")
            sys.stdout.flush()

    sys.stdout.flush()

    print("Columns before removing 'Unnamed':", all_data.columns)

    all_data.columns = all_data.columns.map(str)
    all_data = all_data.loc[:, ~all_data.columns.str.contains('^Unnamed')]

    print("Columns after removing 'Unnamed':", all_data.columns)

    if isinstance(all_data.columns, pd.MultiIndex):
        all_data.columns = [' '.join(col).strip() for col in all_data.columns.values]

    print("Columns after flattening MultiIndex:", all_data.columns)
    
    return all_data

seasons_list = [str(year) for year in range(start, end)]
table_ids = ['mvp', 'roy', 'dpoy', 'smoy', 'mip', 'clutch_poy', 'leading_all_nba', 'leading_all_defense', 'leading_all_rookie', 'coy']

result = scrape_all_seasons_tables(seasons_list, table_ids)

all_award_voting = result

#save
all_award_voting.to_csv('all_award_voting.csv', index=False, encoding="utf-8-sig")

Scraping: [##################################################] 100.00%Columns before removing 'Unnamed': Index(['Rank', 'Player', 'Age', 'Tm', 'First', 'Pts Won', 'Pts Max', 'Share',
       'G', 'MP', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', '3P%', 'FT%', 'WS',
       'WS/48', 'Season', 'award_type', 'DWS', 'DBPM', 'DRtg', '# Tm', 'Pos',
       '1st Tm', '2nd Tm', '3rd Tm', 'Coach', 'W', 'L', 'W/L%'],
      dtype='object')
Columns after removing 'Unnamed': Index(['Rank', 'Player', 'Age', 'Tm', 'First', 'Pts Won', 'Pts Max', 'Share',
       'G', 'MP', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', '3P%', 'FT%', 'WS',
       'WS/48', 'Season', 'award_type', 'DWS', 'DBPM', 'DRtg', '# Tm', 'Pos',
       '1st Tm', '2nd Tm', '3rd Tm', 'Coach', 'W', 'L', 'W/L%'],
      dtype='object')
Columns after flattening MultiIndex: Index(['Rank', 'Player', 'Age', 'Tm', 'First', 'Pts Won', 'Pts Max', 'Share',
       'G', 'MP', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', '3P%', 'FT%', 'WS',
       'WS/48', 'Sea

In [50]:
# Cleaning all award data

coach_data = all_award_voting[all_award_voting['award_type'] == 'coy']
all_award_voting = all_award_voting[all_award_voting['award_type'] != 'coy']
all_award_voting = all_award_voting[all_award_voting['Player'].notna() & (all_award_voting['Player'] != '')]

all_award_voting['1st_team'] = all_award_voting['# Tm'].apply(lambda x: 1 if x in ['1st', '1T'] else 0)
all_award_voting['2nd_team'] = all_award_voting['# Tm'].apply(lambda x: 1 if x in ['2nd', '2T'] else 0)
all_award_voting['3rd_team'] = all_award_voting['# Tm'].apply(lambda x: 1 if x in ['3rd', '3T'] else 0)

all_award_voting = all_award_voting[['Rank', 'Player', 'Tm', 'Share', 'Season', 'award_type', '1st_team', '2nd_team', '3rd_team']]

coach_data = coach_data.rename(columns={'Share': 'coy_share'})
coach_data = coach_data[['Rank', 'Coach', 'Tm', 'coy_share', 'Season']]

all_award_voting['mvp_share'] = all_award_voting.apply(lambda x: x['Share'] if x['award_type'] == 'mvp' else None, axis=1)
all_award_voting['dpoy_share'] = all_award_voting.apply(lambda x: x['Share'] if x['award_type'] == 'dpoy' else None, axis=1)
all_award_voting['roy_share'] = all_award_voting.apply(lambda x: x['Share'] if x['award_type'] == 'roy' else None, axis=1)
all_award_voting['smoy_share'] = all_award_voting.apply(lambda x: x['Share'] if x['award_type'] == 'smoy' else None, axis=1)
all_award_voting['mip_share'] = all_award_voting.apply(lambda x: x['Share'] if x['award_type'] == 'mip' else None, axis=1)
all_award_voting['cpoy_share'] = all_award_voting.apply(lambda x: x['Share'] if x['award_type'] == 'clutch_poy' else None, axis=1)

share_columns = ['mvp_share', 'dpoy_share', 'roy_share', 'smoy_share', 'mip_share', 'cpoy_share']
all_award_voting[share_columns] = all_award_voting[share_columns].apply(pd.to_numeric, errors='coerce')

all_award_voting['leading_all_nba'] = all_award_voting['award_type'].apply(lambda x: 1 if x == 'leading_all_nba' else 0)
all_award_voting['leading_all_defense'] = all_award_voting['award_type'].apply(lambda x: 1 if x == 'leading_all_defense' else 0)
all_award_voting['leading_all_rookie'] = all_award_voting['award_type'].apply(lambda x: 1 if x == 'leading_all_rookie' else 0)

# Get unique award types
award_types = all_award_voting['award_type'].unique()

# Create new columns for each unique award_type combined with '1st_team', '2nd_team', '3rd_team'
for award_type in award_types:
    all_award_voting[f'{award_type}_1st_team'] = all_award_voting.apply(
        lambda row: row['1st_team'] if row['award_type'] == award_type else 0, axis=1
    )
    all_award_voting[f'{award_type}_2nd_team'] = all_award_voting.apply(
        lambda row: row['2nd_team'] if row['award_type'] == award_type else 0, axis=1
    )
    all_award_voting[f'{award_type}_3rd_team'] = all_award_voting.apply(
        lambda row: row['3rd_team'] if row['award_type'] == award_type else 0, axis=1
    )
    all_award_voting[f'{award_type}'] = all_award_voting.apply(
        lambda row: 1 if row['award_type'] == award_type else 0, axis=1
    )

# Create 'count_all_nba', 'count_all_defense', and 'count_all_rookie' columns
all_award_voting['count_all_nba'] = all_award_voting.apply(
    lambda row: 1 if row['leading_all_nba_1st_team'] == 1 or row['leading_all_nba_2nd_team'] == 1 or row['leading_all_nba_3rd_team'] == 1 else 0, axis=1
)
all_award_voting['count_all_defense'] = all_award_voting.apply(
    lambda row: 1 if row['leading_all_defense_1st_team'] == 1 or row['leading_all_defense_2nd_team'] == 1 else 0, axis=1
)
all_award_voting['count_all_rookie'] = all_award_voting.apply(
    lambda row: 1 if row['leading_all_rookie_1st_team'] == 1 or row['leading_all_rookie_2nd_team'] == 1 else 0, axis=1
)

# Drop the specified columns
columns_to_drop = [
    'leading_all_defense_3rd_team', 'leading_all_rookie_3rd_team', 'leading_all_nba', 'leading_all_defense', 'leading_all_rookie',
    'mvp_1st_team', 'mvp_2nd_team', 'mvp_3rd_team', 'mvp', 'roy_1st_team', 'roy_2nd_team', 'roy_3rd_team', 'roy',
    'dpoy_1st_team', 'dpoy_2nd_team', 'dpoy_3rd_team', 'dpoy', 'smoy_1st_team', 'smoy_2nd_team', 'smoy_3rd_team', 'smoy',
    'mip_1st_team', 'mip_2nd_team', 'mip_3rd_team', 'mip', 'clutch_poy_1st_team', 'clutch_poy_2nd_team', 'clutch_poy_3rd_team', 'clutch_poy'
]
all_award_voting.drop(columns=columns_to_drop, inplace=True)

# Drop the 'Share' and 'award_type' columns after creating the new columns
all_award_voting = all_award_voting.drop(columns=['Share', 'award_type'])

all_award_voting['won_mvp'] = all_award_voting.groupby('Season')['mvp_share'].transform(lambda x: (x == x.max()).astype(int))
all_award_voting['won_roy'] = all_award_voting.groupby('Season')['roy_share'].transform(lambda x: (x == x.max()).astype(int))
all_award_voting['won_dpoy'] = all_award_voting.groupby('Season')['dpoy_share'].transform(lambda x: (x == x.max()).astype(int))
all_award_voting['won_smoy'] = all_award_voting.groupby('Season')['smoy_share'].transform(lambda x: (x == x.max()).astype(int))
all_award_voting['won_mip'] = all_award_voting.groupby('Season')['mip_share'].transform(lambda x: (x == x.max()).astype(int))
all_award_voting['won_cpoy'] = all_award_voting.groupby('Season')['cpoy_share'].transform(lambda x: (x == x.max()).astype(int))

coach_data['coy_share'] = pd.to_numeric(coach_data['coy_share'], errors='coerce')
coach_data['won_coy'] = coach_data.groupby('Season')['coy_share'].transform(lambda x: (x == x.max()).astype(int))

all_award_coach_voting = coach_data

#save
all_award_voting.to_csv('all_award_voting.csv', index=False, encoding="utf-8-sig")
all_award_coach_voting.to_csv('all_award_coach_voting.csv', index=False, encoding="utf-8-sig")

### RS | Per Game | Team

In [51]:


def scrape_season(season):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/leagues/NBA_{season}.html'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'per_game-team'})

        if table:
            table_html = str(table)
            df = pd.read_html(StringIO(table_html))[0]

            df['Season'] = season

            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"No table found for {season}")
            return None
    else:
        print(f"Failed to retrieve data for {season}")
        return None

    time.sleep(4)

    return all_data

def scrape_all_seasons(seasons):
    all_data = pd.DataFrame()

    total_seasons = len(seasons)

    for i, season in enumerate(reversed(seasons), 1):
        data = scrape_season(season)

        if data is not None:
            all_data = pd.concat([all_data, data], ignore_index=True)

        completion_percentage = (i / total_seasons) * 100
        sys.stdout.write(f"\rScraping: [{'#' * int(completion_percentage // 2)}{' ' * (50 - int(completion_percentage // 2))}] {completion_percentage:.2f}%")
        sys.stdout.flush()

    sys.stdout.flush()

    all_data = all_data[~all_data['Team'].str.contains('League Average', na=False)]

    all_data['make_playoffs'] = all_data['Team'].str.contains('\*').astype(int)

    all_data['Team'] = all_data['Team'].str.replace('*', '', regex=False)

    all_data = all_data[['Season'] + [col for col in all_data.columns if col != 'Season']]
    all_data = all_data[~all_data['Team'].isin(all_data.columns)]

    return all_data

seasons_list = [str(year) for year in range(start, end)]

result = scrape_all_seasons(seasons_list)

RS_Per_Game_Team = result

#save
RS_Per_Game_Team.to_csv('RS_Per_Game_Team.csv', index=False, encoding="utf-8-sig")

Scraping: [##################################################] 100.00%

### RS | Opp Per Game | Team

In [52]:
def scrape_season(season):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/leagues/NBA_{season}.html'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'per_game-opponent'})

        if table:
            table_html = str(table)
            df = pd.read_html(StringIO(table_html))[0]

            df['Season'] = season

            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"No table found for {season}")
            return None
    else:
        print(f"Failed to retrieve data for {season}")
        return None

    time.sleep(4)

    return all_data

def scrape_all_seasons(seasons):
    all_data = pd.DataFrame()

    total_seasons = len(seasons)

    for i, season in enumerate(reversed(seasons), 1):
        data = scrape_season(season)

        if data is not None:
            all_data = pd.concat([all_data, data], ignore_index=True)

        completion_percentage = (i / total_seasons) * 100
        sys.stdout.write(f"\rScraping: [{'#' * int(completion_percentage // 2)}{' ' * (50 - int(completion_percentage // 2))}] {completion_percentage:.2f}%")
        sys.stdout.flush()

    sys.stdout.flush()

    all_data = all_data[~all_data['Team'].str.contains('League Average', na=False)]

    all_data['make_playoffs'] = all_data['Team'].str.contains('\*').astype(int)

    all_data['Team'] = all_data['Team'].str.replace('*', '', regex=False)

    all_data = all_data[['Season'] + [col for col in all_data.columns if col != 'Season']]
    all_data = all_data[~all_data['Team'].isin(all_data.columns)]

    return all_data

seasons_list = [str(year) for year in range(start, end)]

result = scrape_all_seasons(seasons_list)

columns_to_exclude = ['Season', 'Rk', 'make_playoffs', 'Team']
new_columns = {col: f'opp_{col}' for col in result.columns if col not in columns_to_exclude}
result = result.rename(columns=new_columns)

RS_Opp_Per_Game_Team = result

#save
RS_Opp_Per_Game_Team.to_csv('RS_Opp_Per_Game_Team.csv', index=False)

Scraping: [##################################################] 100.00%

### RS | Advanced | Team

In [53]:


def scrape_season(season):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/leagues/NBA_{season}.html'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'advanced-team'})

        if table:
            html_str = str(table)
            df = pd.read_html(StringIO(html_str))[0]

            df['Season'] = season

            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"No table found for {season}")
            return None
    else:
        print(f"Failed to retrieve data for {season}")
        return None

    time.sleep(4)

    return all_data

def scrape_all_seasons(seasons_list):
    all_data = pd.DataFrame()

    for season in seasons_list:
        data = scrape_season(season)

        if data is not None:
            data['Season'] = season

            all_data = pd.concat([all_data, data], ignore_index=True)

            print(f"Scraping data for season {season}")

    return all_data

seasons_list = [str(year) for year in range(start, end)]

result = scrape_all_seasons(seasons_list)

RS_Advanced_Team = result

#cleaning
RS_Advanced_Team.columns = RS_Advanced_Team.iloc[0]
RS_Advanced_Team = RS_Advanced_Team[1:]
RS_Advanced_Team.columns.name = None
RS_Advanced_Team.columns = RS_Advanced_Team.columns.astype(str)
RS_Advanced_Team = RS_Advanced_Team.loc[:, ~RS_Advanced_Team.columns.str.contains('^Unnamed')]

RS_Advanced_Team = RS_Advanced_Team[RS_Advanced_Team['Team'] != 'League Average']
RS_Advanced_Team['make_playoffs'] = RS_Advanced_Team['Team'].apply(lambda x: 1 if '*' in x else 0)

RS_Advanced_Team['season'] = RS_Advanced_Team.iloc[:, RS_Advanced_Team.columns.get_loc('Attend./G') + 1].astype(str)
RS_Advanced_Team.drop(columns=RS_Advanced_Team.columns[RS_Advanced_Team.columns.get_loc('Attend./G') + 1], inplace=True)

RS_Advanced_Team['Rk'] = RS_Advanced_Team['Rk'].astype(float).round().astype(int).astype(str)
RS_Advanced_Team['season'] = RS_Advanced_Team['season'].astype(float).round().astype(int).astype(str)

RS_Advanced_Team['Team'] = RS_Advanced_Team['Team'].str.replace('*', '')
RS_Advanced_Team['overall_record'] = RS_Advanced_Team['W'].astype(float) / (RS_Advanced_Team['W'].astype(float) + RS_Advanced_Team['L'].astype(float))

RS_Advanced_Team['rk_season'] = RS_Advanced_Team.groupby('season')['overall_record'].rank(ascending=False)
RS_Advanced_Team['rk_season'] = RS_Advanced_Team['rk_season'].fillna(0).astype(int)

for idx in [17, 18, 19, 20]:
    RS_Advanced_Team.columns.values[idx] = 'offensive_' + RS_Advanced_Team.columns[idx]

for idx in [21, 22, 23, 24]:
    RS_Advanced_Team.columns.values[idx] = 'defensive_' + RS_Advanced_Team.columns[idx]

#save    
RS_Advanced_Team.to_csv('RS_Advanced_Team.csv',index=False, encoding="utf-8-sig")

Scraping data for season 2024


### PO | Advanced | Team

In [55]:
def scrape_season(season):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/playoffs/NBA_{season}.html'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'advanced-team'})

        if table:
            html_str = str(table)
            df = pd.read_html(StringIO(html_str))[0]

            df['Season'] = season

            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"No table found for {season}")
            return None
    else:
        print(f"Failed to retrieve data for {season}")
        return None

    time.sleep(4)

    return all_data

def scrape_all_seasons(seasons_list):
    all_data = pd.DataFrame()

    for season in seasons_list:
        data = scrape_season(season)

        if data is not None:
            data['Season'] = season

            all_data = pd.concat([all_data, data], ignore_index=True)

            print(f"Scraping data for season {season}")

    return all_data

seasons_list = [str(year) for year in range(start, end)]

result = scrape_all_seasons(seasons_list)

PO_Advanced_Team = result

# Create a backup of the original DataFrame
PO_Advanced_Team_backup = copy.deepcopy(PO_Advanced_Team)
    
# Old code
PO_Advanced_Team.columns = PO_Advanced_Team.iloc[0]
PO_Advanced_Team = PO_Advanced_Team[1:]
    
PO_Advanced_Team.columns.name = None
    
PO_Advanced_Team.columns = PO_Advanced_Team.columns.astype(str)
    
PO_Advanced_Team = PO_Advanced_Team[PO_Advanced_Team['Tm'] != 'League Average']
    
PO_Advanced_Team = PO_Advanced_Team.loc[:, ~PO_Advanced_Team.columns.str.contains('^Unnamed')]
    
PO_Advanced_Team.rename(columns={'nan': 'season'}, inplace=True)
    
PO_Advanced_Team['Rk'] = PO_Advanced_Team['Rk'].fillna(-1).astype(float).round().astype(int).astype(str)
    
PO_Advanced_Team['season'] = PO_Advanced_Team['season'].astype(float).round().astype(int).astype(str)
    
PO_Advanced_Team['W'] = PO_Advanced_Team['W'].astype(float)
    
PO_Advanced_Team['season'] = PO_Advanced_Team['season'].astype(str)
    
PO_Advanced_Team['champion_share'] = PO_Advanced_Team['W'] / PO_Advanced_Team.groupby('season')['W'].transform('max')
    
for idx in [15, 16, 17, 18]:
    PO_Advanced_Team.columns.values[idx] = 'offensive_' + PO_Advanced_Team.columns[idx]
    
for idx in [19, 20, 21, 22]:
    PO_Advanced_Team.columns.values[idx] = 'defensive_' + PO_Advanced_Team.columns[idx]
    
#save    
PO_Advanced_Team.to_csv('RS_Advanced_Team.csv',index=False, encoding="utf-8-sig")

Scraping data for season 2024


### RS | Schedule/Results | Team

In [57]:

def scrape_season(season):
    all_data = pd.DataFrame()
    months = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june', 'july']

    for month in months:
        url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games-{month}.html'
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find('table', {'id': 'schedule'})

            if table:
                html_str = str(table)
                df = pd.read_html(StringIO(html_str))[0]

                df['Season'] = season

                all_data = pd.concat([all_data, df], ignore_index=True)
            else:
                print(f"No table found for {season} in {month}")
        else:
            print(f"Failed to retrieve data for {season} in {month}")

        time.sleep(4)

    return all_data

def scrape_all_seasons():
    all_data = pd.DataFrame()

    for season in range(start, end):
        data = scrape_season(str(season))

        if data is not None:
            data['Season'] = season

            all_data = pd.concat([all_data, data], ignore_index=True)

            print(f"Scraping data for season {season}")

    return all_data

result = scrape_all_seasons()

RS_Schedule_Team = result

#save
RS_Schedule_Team.to_csv('RS_Schedule_Team.csv', index=False, encoding="utf-8-sig")

Failed to retrieve data for 2024 in july
Scraping data for season 2024


### PO | Schedule/Results | Team

In [58]:

def scrape_season(season):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/playoffs/NBA_{season}_games.html'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'schedule'})

        if table:
            html_str = str(table)
            df = pd.read_html(StringIO(html_str))[0]

            df['Season'] = season

            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"No table found for {season}")
    else:
        print(f"Failed to retrieve data for {season}")

    time.sleep(4)

    return all_data

def scrape_all_seasons():
    all_data = pd.DataFrame()

    for season in range(start, end):
        data = scrape_season(str(season))

        if data is not None:
            data['Season'] = season

            all_data = pd.concat([all_data, data], ignore_index=True)

            print(f"Scraping data for season {season}")

    return all_data

result = scrape_all_seasons()

PO_Schedule_Team = result
PO_Schedule_Team.to_csv('PO_Schedule_Team.csv', index=False, encoding="utf-8-sig")

#=======================================================================================================

#clean all schedule data

dtype = {
    'Date': str,
    'Start (ET)': str,
    'Visitor/Neutral': str,
    'PTS': float,
    'Home/Neutral': str,
    'PTS.1': float,
    'Unnamed: 7': str,  # Replace 'Unnamed: 7' with the actual column name if known
    'Attend.': float,
    'LOG': str,
    'Notes': str
}
#btw play in games are NOT counted for both

#=======================================================================================================

#Cleaning regular season schedule data

RS_Schedule_Team['matchup'] = RS_Schedule_Team['Visitor/Neutral'] + '_' + RS_Schedule_Team['Home/Neutral']

RS_Schedule_Team['game_id'] = RS_Schedule_Team['Date'] + '_' + RS_Schedule_Team['matchup']

RS_Schedule_Team = RS_Schedule_Team.loc[:, ~RS_Schedule_Team.columns.str.contains('^Unnamed')]

RS_Schedule_Team = RS_Schedule_Team[~RS_Schedule_Team['Notes'].str.contains('Play-In Game', na=False)]

columns_to_drop = ['Notes', 'Arena', 'LOG', 'Attend.']
RS_Schedule_Team = RS_Schedule_Team.drop(columns=columns_to_drop, errors='ignore')

RS_Schedule_Team = RS_Schedule_Team.rename(columns={'PTS': 'v/n_pts', 'PTS.1': 'h/n_pts'})

#=======================================================================================================

#Cleaning playoff schedule data

PO_Schedule_Team['matchup'] = PO_Schedule_Team['Visitor/Neutral'] + '_' + PO_Schedule_Team['Home/Neutral']

PO_Schedule_Team['game_id'] = PO_Schedule_Team['Date'] + '_' + PO_Schedule_Team['matchup']

PO_Schedule_Team = PO_Schedule_Team.loc[:, ~PO_Schedule_Team.columns.str.contains('^Unnamed')]

columns_to_drop = ['Notes', 'Arena', 'LOG', 'Attend.']
PO_Schedule_Team = PO_Schedule_Team.drop(columns=columns_to_drop, errors='ignore')

PO_Schedule_Team = PO_Schedule_Team.rename(columns={'PTS': 'v/n_pts', 'PTS.1': 'h/n_pts'})

#=======================================================================================================

RS_Schedule_Team = RS_Schedule_Team[~RS_Schedule_Team['game_id'].isin(PO_Schedule_Team['game_id'])]

#save
RS_Schedule_Team.to_csv('RS_Schedule_Team.csv', index=False, encoding="utf-8-sig")
PO_Schedule_Team.to_csv('PO_Schedule_Team.csv', index=False, encoding="utf-8-sig")

Scraping data for season 2024


### Other | Pre-Season Odds | Team

In [60]:
def scrape_season(season):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_preseason_odds.html'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'NBA_preseason_odds'})

        if table:
            html_str = str(table)
            df = pd.read_html(StringIO(html_str))[0]

            df['Season'] = season

            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"No table found for {season}")
    else:
        print(f"Failed to retrieve data for {season}")

    time.sleep(4)

    return all_data

def scrape_all_seasons(seasons_list):
    all_data = pd.DataFrame()

    for season in seasons_list:
        data = scrape_season(season)

        if data is not None:
            data['Season'] = season

            all_data = pd.concat([all_data, data], ignore_index=True)

            print(f"Scraping data for season {season}")

    return all_data

seasons_list = [str(year) for year in range(start, end)]

result = scrape_all_seasons(seasons_list)

PSO_Team = result

#=================================================================================================

#cleaning pso team data

PSO_Team = PSO_Team.loc[:, ~PSO_Team.columns.str.contains('^Unnamed')]

PSO_Team['overall_record_o/u'] = PSO_Team['Result'].str.contains(r'\(over\)', na=False).astype(int)

PSO_Team = PSO_Team.rename(columns={'W-L O/U': 'projected_overall_record'})

def calculate_overall_record(result):
    if pd.isna(result):
        return None
    try:
        record = result.split(' ')[0]  #rec part
        wins, losses = map(int, record.split('-'))
        return wins / (wins + losses)
    except Exception as e:
        return None

PSO_Team['overall_record'] = PSO_Team['Result'].apply(calculate_overall_record)

PSO_Team['overall_record'] = (PSO_Team['overall_record'] * 100).round(1)

PSO_Team = PSO_Team.drop(columns=['Result'])

#save
PSO_Team.to_csv('PSO_Team.csv', index=False, encoding="utf-8-sig")

Scraping data for season 2024


### RS | Expanded Standings | Team

In [62]:

def scrape_season(season):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_standings.html#all_expanded_standings'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))
        
        for comment in comments:
            if 'expanded_standings' in comment:
                comment_soup = BeautifulSoup(comment, 'html.parser')
                table = comment_soup.find('table', {'id': 'expanded_standings'})
                
                if table:
                    
                    df = pd.read_html(StringIO(str(table)))[0]

                    df['Season'] = season

                    all_data = pd.concat([all_data, df], ignore_index=True)
                    break
        else:
            print(f"No table found for {season}")
            return None
    else:
        print(f"Failed to retrieve data for {season}")
        return None

    time.sleep(4)

    return all_data

def scrape_all_seasons():
    all_data = pd.DataFrame()

    for season in range(start, end):
        data = scrape_season(str(season))

        if data is not None:
            data['Season'] = season

            all_data = pd.concat([all_data, data], ignore_index=True)

            print(f"Scraping data for season {season}")

    return all_data

result = scrape_all_seasons()

RS_exp_stand_Team = result

#==========================================================================================================================

#clean expanded standings data

RS_exp_stand_Team = pd.read_csv("RS_exp_stand_Team.csv")

RS_exp_stand_Team.columns = RS_exp_stand_Team.iloc[0]
RS_exp_stand_Team = RS_exp_stand_Team[1:]

RS_exp_stand_Team.reset_index(drop=True, inplace=True)

new_column_headers = [
    'Rk', 'Team', 'overall_record', 'home_rec', 'road_rec', 'neu_rec', 'cen_div_rec', 'e_div_rec',
    'w_div_rec', '3pt_or_less_rec', '10pt_or_more_rec', 'oct_mon_rec', 'nov_mon_rec', 'dec_mon_rec',
    'jan_mon_rec', 'feb_mon_rec', 'mar_mon_rec', 'season', 'pre_all_star_rec', 'post_all_star_rec',
    'e_conf_rec', 'w_conf_rec', 'atl_div_rec', 'midw_div_rec', 'pac_div_rec', 'apr_mon_rec',
    'may_mon_rec', 'se_div_rec', 'nw_div_rec', 'sw_div_rec', 'jul_mon_rec', 'aug_mon_rec'
]

if len(RS_exp_stand_Team.columns) >= len(new_column_headers):
    RS_exp_stand_Team.columns = new_column_headers
else:
    print("The DataFrame does not have enough columns to rename according to the provided headers.")

def calculate_winning_percentage(record):
    try:
        wins, losses = map(int, record.split('-'))
        percentage = wins / (wins + losses)
        return percentage
    except:
        return None

for column in RS_exp_stand_Team.columns:
    if column not in ['Rk', 'Team', 'season']:
        RS_exp_stand_Team[column] = RS_exp_stand_Team[column].apply(calculate_winning_percentage)


#save
RS_exp_stand_Team.to_csv('RS_exp_stand_Team.csv', index=False, encoding="utf-8-sig")

Scraping data for season 1950
Scraping data for season 1951
Scraping data for season 1952
Scraping data for season 1953
Scraping data for season 1954
Scraping data for season 1955
Scraping data for season 1956
Scraping data for season 1957
Scraping data for season 1958
Scraping data for season 1959
Scraping data for season 1960
Scraping data for season 1961
Scraping data for season 1962
Scraping data for season 1963
Scraping data for season 1964
Scraping data for season 1965
Scraping data for season 1966
Scraping data for season 1967
Scraping data for season 1968
Scraping data for season 1969
Scraping data for season 1970
Scraping data for season 1971
Scraping data for season 1972
Scraping data for season 1973
Scraping data for season 1974
Scraping data for season 1975
Scraping data for season 1976
Scraping data for season 1977
Scraping data for season 1978
Scraping data for season 1979
Scraping data for season 1980
Scraping data for season 1981
Scraping data for season 1982
Scraping d

### Season | Coaches

In [65]:


def scrape_season(season):
    all_data = pd.DataFrame()

    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_coaches.html'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'NBA_coaches'})

        if table:
            df = pd.read_html(io.StringIO(str(table)))[0]

            df['Season'] = season

            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"No table found for {season}")
    else:
        print(f"Failed to retrieve data for {season}")

    time.sleep(4)

    return all_data

def scrape_all_seasons():
    all_data = pd.DataFrame()

    for season in range(start, end):  
        data = scrape_season(str(season))

        if data is not None:
            data['Season'] = season
            all_data = pd.concat([all_data, data], ignore_index=True)

            print(f"Scraping data for season {season}")

    return all_data

result = scrape_all_seasons()

coaches_season = result

#==============================================================================================================

#clean coach data

coaches_season = coaches_season.iloc[2:].reset_index(drop=True)

columns_to_drop = [2, 5, 16]
coaches_season.drop(coaches_season.columns[columns_to_drop], axis=1, inplace=True)

new_column_headers = [
    'Coach', 'Tm', 'sea_w/franc_count', 'sea_overall_count', 'rs_curr_sea_G', 'rs_curr_sea_W', 'rs_curr_sea_L',
    'rs_w/franc_G', 'rs_w/franc_W', 'rs_w/franc_L', 'rs_career_G', 'rs_career_W',
    'rs_career_L', 'rs_career_W%', 'po_curr_sea_G', 'po_curr_sea_W', 'po_curr_sea_L',
    'po_w/franc_G', 'po_w/franc_W', 'po_w/franc_L', 'po_career_G', 'po_career_W', 'po_career_L', 'season'
]

coaches_season.columns = new_column_headers

coaches_season['season'] = coaches_season['season'].astype(float).round().astype(int)

#save
coaches_season.to_csv('coaches_season.csv', index=False)

Scraping data for season 2024


### Final cleaning

In [20]:
#cleaned files
PO_Schedule_Team = pd.read_csv("PO_Schedule_Team.csv")
RS_Schedule_Team = pd.read_csv("RS_Schedule_Team.csv")

custom_team_season_index = pd.read_csv("custom_team_season_index.csv")
custom_team_franchise_index = pd.read_csv("custom_team_franchise_index.csv")

#clean up files
coaches_season = pd.read_csv("coaches_season.csv")
RS_exp_stand_Team = pd.read_csv("RS_exp_stand_Team.csv") 
PSO_Team = pd.read_csv("PSO_Team.csv")

PO_Advanced_Team = pd.read_csv("PO_Advanced_Team.csv")
RS_Advanced_Team = pd.read_csv("RS_Advanced_Team.csv")
RS_Opp_Per_Game_Team = pd.read_csv("RS_Opp_Per_Game_Team.csv") 
RS_Per_Game_Team = pd.read_csv("RS_Per_Game_Team.csv")

all_award_coach_voting = pd.read_csv("all_award_coach_voting.csv") 
all_award_voting = pd.read_csv("all_award_voting.csv")

PO_Totals_Player = pd.read_csv("PO_Totals_Player.csv")
PO_Advanced_Player = pd.read_csv("PO_Advanced_Player.csv")
PO_Per_Game_Player = pd.read_csv("PO_Per_Game_Player.csv")

RS_Totals_Player = pd.read_csv("RS_Totals_Player.csv")
RS_Advanced_Player = pd.read_csv("RS_Advanced_Player.csv") 
RS_Per_Game_Player = pd.read_csv("RS_Per_Game_Player.csv")

RS_exp_stand_Team = pd.read_csv("RS_exp_stand_Team.csv")

In [21]:
#cleaning and merging, add index columns to datasets

#giving these dfs the team_id column from the index
coaches_season = pd.merge(coaches_season, custom_team_franchise_index[['Tm', 'team_id']], on='Tm', how='left')
RS_exp_stand_Team = pd.merge(RS_exp_stand_Team, custom_team_franchise_index[['Team', 'team_id']], on='Team', how='left')
PSO_Team = pd.merge(PSO_Team, custom_team_franchise_index[['Team', 'team_id']], on='Team', how='left')
all_award_coach_voting = pd.merge(all_award_coach_voting, custom_team_franchise_index[['Tm', 'team_id']], on='Tm', how='left')

#=================================================================================================================================

#remove "*" from all rows in the 'Player' column
all_award_voting['Player'] = all_award_voting['Player'].str.replace('*', '')

#drop all rows that contain "Player" in the 'Player' column
all_award_voting = all_award_voting[all_award_voting['Player'] != 'Player']

#=================================================================================================================================

#clean po advanced team 

#if there is a NA in column Team fill that with value/text in column Tm
PO_Advanced_Team['Team'] = PO_Advanced_Team['Team'].fillna(PO_Advanced_Team['Tm'])

#then delete column Tm
PO_Advanced_Team.drop(columns=['Tm'], inplace=True)

#delete row(s) in 'League Average' in column Team
PO_Advanced_Team = PO_Advanced_Team[PO_Advanced_Team['Team'] != 'League Average']

#give team_id column to po advanced team
PO_Advanced_Team = pd.merge(PO_Advanced_Team, custom_team_franchise_index[['Team', 'team_id']], on='Team', how='left')

#make sure team_id is a whole number in dataframe
PO_Advanced_Team['team_id'] = PO_Advanced_Team['team_id'].round().astype(int)

#add new feature champion, if champion share == 1 then give it 1
PO_Advanced_Team['champion'] = PO_Advanced_Team['champion_share'].apply(lambda x: 1 if x == 1 else 0)

#=================================================================================================================================

#clean rs advanced team 

#give rs advanced team the team_id column
RS_Advanced_Team = pd.merge(RS_Advanced_Team, custom_team_franchise_index[['Team', 'team_id']], on='Team', how='left')

#=================================================================================================================================

#clean rs opp per game team

#give rs opp per game team the team id column
RS_Opp_Per_Game_Team = pd.merge(RS_Opp_Per_Game_Team, custom_team_franchise_index[['Team', 'team_id']], on='Team', how='left')


#clean rs per game team

#=================================================================================================================================

#give rs per game the team id column
RS_Per_Game_Team = pd.merge(RS_Per_Game_Team, custom_team_franchise_index[['Team', 'team_id']], on='Team', how='left')

In [22]:
#function to rename any column called 'Season' to 'season in our dfs for consistency
def rename_season_column(df):
    if 'Season' in df.columns:
        df.rename(columns={'Season': 'season'}, inplace=True)
    elif 'season' in df.columns:
        df.rename(columns={'season': 'season'}, inplace=True)
    return df

#dfs to process
dfs = {
    'coaches_season': coaches_season,
    'RS_exp_stand_Team': RS_exp_stand_Team,
    'PSO_Team': PSO_Team,
    'all_award_coach_voting': all_award_coach_voting,
    'all_award_voting': all_award_voting,
    'PO_Totals_Player': PO_Totals_Player,
    'PO_Advanced_Player': PO_Advanced_Player,
    'PO_Per_Game_Player': PO_Per_Game_Player,
    'PO_Advanced_Team': PO_Advanced_Team,
    'RS_Advanced_Team': RS_Advanced_Team,
    'RS_Opp_Per_Game_Team': RS_Opp_Per_Game_Team,
    'RS_Per_Game_Team': RS_Per_Game_Team,
    'RS_Totals_Player': RS_Totals_Player,
    'RS_Advanced_Player': RS_Advanced_Player,
    'RS_Per_Game_Player': RS_Per_Game_Player
}

#apply
for name, df in dfs.items():
    dfs[name] = rename_season_column(df)

In [23]:
def update_teams_and_filter(df, po_df):
    #make sure all 'season' column in those dfs are rounded to whole value
    df['season'] = df['season'].astype(int)
    po_df['season'] = po_df['season'].astype(int)
    
    #merging to to get the Tm_po column from po per game player df so we have a column indicating if were that players is playing on team come playoff time
    merged_df = df.merge( 
        po_df[['Player', 'season', 'Tm']], 
        on=['Player', 'season'], 
        how='left',
        suffixes=('', '_po')
    )

    #create team update column with the values from Tm_PO or fall back to the original values in Tm
    merged_df['team_update'] = merged_df['Tm_po'].combine_first(merged_df['Tm'])

    #drop the extra Tm po column used for merging
    merged_df.drop(columns=['Tm_po'], inplace=True)

    #rename the 'Tm' column to 'team_before_td'
    merged_df.rename(columns={'Tm': 'team_before_td'}, inplace=True)

    #group by 'Player' and 'season', and select the row with the maximum value in 'G'
    filtered_df = merged_df.loc[merged_df.groupby(['Player', 'season'])['G'].idxmax()]

    #create the 'team_after_td' column based on the specified criteria
    filtered_df['team_after_td'] = filtered_df.apply(
        lambda row: row['team_update'] if row['team_before_td'] == 'TOT' else row['team_before_td'], axis=1
    )

    #drop the team update column
    filtered_df.drop(columns=['team_update'], inplace=True)

    return filtered_df

#apply function to these dfs
RS_Per_Game_Player = update_teams_and_filter(RS_Per_Game_Player, PO_Per_Game_Player)
RS_Advanced_Player = update_teams_and_filter(RS_Advanced_Player, PO_Per_Game_Player)
RS_Totals_Player = update_teams_and_filter(RS_Totals_Player, PO_Per_Game_Player)

#============================================================================================================================

#cleaning the award data

#list of columns to average- JA
columns_to_average = [
    '1st_team', '2nd_team', '3rd_team', 'mvp_share', 'dpoy_share', 'roy_share', 'smoy_share', 
    'mip_share', 'cpoy_share', 'leading_all_nba_1st_team', 'leading_all_nba_2nd_team', 
    'leading_all_nba_3rd_team', 'leading_all_defense_1st_team', 'leading_all_defense_2nd_team', 
    'leading_all_rookie_1st_team', 'leading_all_rookie_2nd_team', 'count_all_nba', 
    'count_all_defense', 'count_all_rookie', 'won_mvp', 'won_roy', 'won_dpoy', 
    'won_smoy', 'won_mip', 'won_cpoy'
]

#group by 'Player' and 'season' and calculate the mean of the specified columns- JA
all_award_voting = all_award_voting.groupby(['Player', 'season'])[columns_to_average].mean().reset_index()

#columns to check and convert values greater than 0 to 1- JA 
columns_to_convert = [
    '1st_team', '2nd_team', '3rd_team', 'leading_all_nba_1st_team', 'leading_all_nba_2nd_team', 
    'leading_all_nba_3rd_team', 'leading_all_defense_1st_team', 'leading_all_defense_2nd_team', 
    'leading_all_rookie_1st_team', 'leading_all_rookie_2nd_team', 'count_all_nba', 
    'count_all_defense', 'count_all_rookie', 'won_mvp', 'won_roy', 'won_dpoy', 
    'won_smoy', 'won_mip', 'won_cpoy'
]

#convert values greater than 0 to 1- JA
for col in columns_to_convert:
    all_award_voting[col] = all_award_voting[col].apply(lambda x: 1 if x > 0 else 0)

#### Final raw data csv's | Cleaned

In [24]:
coaches_season.to_csv('coaches_season.csv', index=False, encoding="utf-8-sig")
RS_exp_stand_Team.to_csv('RS_exp_stand_team.csv', index=False, encoding="utf-8-sig")
PSO_Team.to_csv('PSO_Team.csv', index=False, encoding="utf-8-sig")

all_award_coach_voting.to_csv('all_award_coach_voting.csv', index=False, encoding="utf-8-sig")
all_award_voting.to_csv('all_award_voting.csv', index=False, encoding="utf-8-sig")

PO_Totals_Player.to_csv('PO_Totals_Player.csv', index=False, encoding="utf-8-sig")
PO_Advanced_Player.to_csv('PO_Advanced_Player.csv', index=False, encoding="utf-8-sig")
PO_Per_Game_Player.to_csv('PO_Per_Game_Player.csv', index=False, encoding="utf-8-sig")

PO_Advanced_Team.to_csv('PO_Advanced_Team.csv', index=False, encoding="utf-8-sig")

RS_Advanced_Team.to_csv('RS_Advanced_Team.csv', index=False, encoding="utf-8-sig")
RS_Opp_Per_Game_Team.to_csv('RS_Opp_Per_Game_Team.csv', index=False, encoding="utf-8-sig")
RS_Per_Game_Team.to_csv('RS_Per_Game_Team.csv', index=False, encoding="utf-8-sig")

RS_Totals_Player.to_csv('RS_Totals_Player.csv', index=False, encoding="utf-8-sig")
RS_Advanced_Player.to_csv('RS_Advanced_Player.csv', index=False, encoding="utf-8-sig")
RS_Per_Game_Player.to_csv('RS_Per_Game_Player.csv', index=False, encoding="utf-8-sig")